In [5]:
# %load ~/helpers.py
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings("ignore", module="matplotlib")

# Styles
jtplot.style('oceans16')
jtplot.style('grade3', context='paper', fscale=1.4)
jtplot.figsize(aspect=1.2)
jtplot.style(ticks=False, grid=False)

# Debug
from IPython.core.debugger import set_trace


In [64]:
size = 4
def _slide_left(b, left=True):
    """Slide left once, without condensing"""
    b = np.copy(b)
    for y in range(size):
        for x in range(size): # Left side can't slide left
            offset = -1 if left else 1
            if x+offset < 0 or x+offset >= size:
                continue
            
            left_val = b[y, x+offset]
            if left_val == 0: # Only slide if zero to the left
                b[y, x+offset] = b[y,x]
                b[y,x] = 0 # Leave a zero where we previously were
    return b

def _condense_left(b, left=True):
    """Condense iff your direct left side matches you, leaves a zero in previous location"""
    b = np.copy(b)
    for y in range(size):
        for x in range(size): # Left side can't condense
            offset = -1 if left else 1
            if x+offset < 0 or x+offset >= size:
                continue
            left_val = b[y, x+offset]
            if left_val ==  b[y,x]:
                b[y, x+offset] = left_val * 2
                b[y,x] = 0 # Leave a zero where we previously were            
    return b

def _slide_then_condense(b, left=True):
    """Takes a 4x4 matrix and shifts all values to the left, condensing once"""
    for _ in range(size): # Slide all our elements
        b = _slide_left(b, left)
    b = _condense_left(b, left)
    for _ in range(size // 2): # Again slide all our elements that might now be free
        b = _slide_left(b, left)    
    return b

def left(b):
    return _slide_then_condense(b, True)

def right(b):
    return _slide_then_condense(b, False)


In [61]:
# Test condensing
full_top_row = np.zeros([4,4])
full_top_row[0,:] = 1

condensed = np.zeros([4,4])
condensed[0,0] = 2
condensed[0,1] = 2

np.testing.assert_array_equal(left(full_top_row), condensed)

# Test condensing three
three_top = np.zeros([4,4])
three_top[0,:-1] = 1

condensed = np.zeros([4,4])
condensed[0,0] = 2
condensed[0,1] = 1
np.testing.assert_array_equal(left(three_top), condensed)

# Test 4 2 2 => 4 4
four_two_two = np.zeros([4,4])
four_two_two[0,:-1] = 2
four_two_two[0,0] = 4

condensed = np.zeros([4,4])
condensed[0,0] = 4
condensed[0,1] = 4
np.testing.assert_array_equal(left(four_two_two), condensed)


In [62]:
# Test empty
empty = np.zeros([4,4])
np.testing.assert_array_equal(left(empty), empty)

# Test basic sliding movement
right_one = np.zeros([4,4])
right_one[0,3] = 1

left_one = np.zeros([4,4])
left_one[0,0] = 1

np.testing.assert_array_equal(left(left_one), left_one)
np.testing.assert_array_equal(left(right_one), left_one)

# Test multiple rows
right_ones = np.zeros([4,4])
right_ones[0,3] = 1
right_ones[3,3] = 1
left_ones = np.zeros([4,4])
left_ones[0,0] = 1
left_ones[3,0] = 1

np.testing.assert_array_equal(left(left_ones), left_ones)
np.testing.assert_array_equal(left(right_ones), left_ones)


In [67]:
ones = np.ones([4,4])
print(ones)

print()
print("left")
print(left(ones))

print()
print("left")
print(left(left(ones)))


print()
print("right")
print(right(left(left(ones))))

[[ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]]

left
[[ 2.  2.  0.  0.]
 [ 2.  2.  0.  0.]
 [ 2.  2.  0.  0.]
 [ 2.  2.  0.  0.]]

left
[[ 4.  0.  0.  0.]
 [ 4.  0.  0.  0.]
 [ 4.  0.  0.  0.]
 [ 4.  0.  0.  0.]]

right
[[ 0.  0.  0.  4.]
 [ 0.  0.  0.  4.]
 [ 0.  0.  0.  4.]
 [ 0.  0.  0.  4.]]
